## **Bacteremia Model Analysis**

Due to confidentiality agreements with the hospital, original data cannot be shared. However, model outputs such as predicted probabilities and validation labels are retained for analysis. In addition, the process of selecting class weights is demonstrated through complete code examples, even though this section does not retain any output for further evaluation.

---

### Summary
1. **Settings:**
- Environment setup and configurations.

1. **Assign Variables:**
- Define input features, labels, and group IDs for modeling.

1. **Class weight:**
- Demonstrates code for tuning class weights. 
- No output is stored.

1. **CBC/CPD vs CBC vs CPD:**
- Compare feature sets (All, CBC, CPD). 
- Probabilities and labels are stored.
- Delong's test (*Executable*)

1. **Imputation method:**
- Test imputation methods (mean, zero, median, knn). 
- Outputs are stored.
- Delong's test (*Executable*)


## **Library**

In [ ]:
# ========= Basic Utilities and System Setup =========
import os
import sys

# ========= Data Handling =========
import pandas as pd
import numpy as np  

# ========= Machine Learning =========
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import label_binarize
from catboost import CatBoostClassifier

# ========= Tools and Visualization =========
from tqdm import tqdm
from joblib import dump, load

# ========= Custom Modules =========
# Add custom module path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
sys.path.insert(0, src_path)

from config import data_dir, cache_dir
from model_utils import (
    custom_classify,
    get_metrics,
    imputer,
    scaling,
    find_classwise_thresholds_by_youden
)

from compare_auc_delong_xu import delong_roc_test


## **Settings**

In [2]:
random_seed= 564
impute_method = 'mean'
class_weights = {0: 1, 1: 1, 2:1}

## **Assign Variables**

In [3]:
# Load data 
data_train = pd.read_csv(
    os.path.join(data_dir, 'CMUH_GS_2021_triage.csv'),
)

data_test = pd.read_csv(
    os.path.join(data_dir, 'CMUH_GS_2023_triage.csv'),
)

# Load external validation datasets from AN and WMH hospitals
data_ANH = pd.read_csv(
    os.path.join(data_dir, 'AN_GS_2023.csv'),
)

data_WMH = pd.read_csv(
    os.path.join(data_dir, 'WK_GS_2023.csv'),
)

In [4]:
y = data_train.loc[:,'final_label']
x = data_train.iloc[:,6:93]

y_test = data_test.loc[:,'final_label']
x_test = data_test.iloc[:,6:93]

y_wk = data_WMH.loc[:,'final_label']
x_wk = data_WMH.iloc[:,6:93]

y_an = data_ANH.loc[:,'final_label']
x_an = data_ANH.iloc[:,6:93]

x_base = data_train[['SIRS']]
x_test_base = data_test[['SIRS']]

In [26]:
from collections import Counter

# List of label arrays and their corresponding names
label_sets = [
    ("CMUH development", y),
    ("CMUH validation", y_test),
    ("WMH validation", y_wk),
    ("ANH validation", y_an)
]

for name, y_labels in label_sets:
    element_counts = Counter(y_labels)
    total_elements = len(y_labels)

    print(f"\n=== {name} ===")
    print(f"Total cases: {total_elements}")
    
    for cls in [0, 1, 2]:
        count = element_counts[cls]
        proportion = count / total_elements
        print(f"Class {cls}: {count} ({proportion:.2%})")



=== CMUH development ===
Total cases: 28503
Class 0: 25534 (89.58%)
Class 1: 2174 (7.63%)
Class 2: 795 (2.79%)

=== CMUH validation ===
Total cases: 15801
Class 0: 14387 (91.05%)
Class 1: 997 (6.31%)
Class 2: 417 (2.64%)

=== WMH validation ===
Total cases: 2632
Class 0: 2415 (91.76%)
Class 1: 173 (6.57%)
Class 2: 44 (1.67%)

=== ANH validation ===
Total cases: 3811
Class 0: 3556 (93.31%)
Class 1: 185 (4.85%)
Class 2: 70 (1.84%)


## **Class weight**

In [20]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)
train_idx, val_idx = next(gss.split(x, y, groups=data_train['ID']))

x_train, x_val = x.iloc[train_idx], x.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

y_val_bin = label_binarize(y_val, classes=[0, 1, 2])

x_train_imp, m_mean = imputer(x_train, impute_method, train=True)
x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

x_val_imp = imputer(x_val, impute_method, False, m_mean)
x_val_scale, train_scaler = scaling (x_val_imp, train_scaler, train= False)


#### **Proposed Catboost model**

In [22]:
import itertools

# Define possible weight for Gram negative (class 1) is a_values, gram positive (class 2) is b_values
a_values = [1, 2, 4, 8]
b_values = [1, 4, 8, 16, 32]

results = []  # Store performance metrics for each (a, b) combination

# Iterate over all combinations of (a, b) using Cartesian product
for a, b in tqdm(itertools.product(a_values, b_values), total=len(a_values) * len(b_values)):
    
    # Set class weights:  Gram negative (class 1) is a, gram positive (class 2) is b
    class_weights = {0: 1, 1: a, 2: b}

    model = CatBoostClassifier(verbose=False, class_weights=class_weights)
    model.fit(x_train_scale, y_train)

    y_prob = model.predict_proba(x_val_scale)

    # Apply Youden's index to determine optimal thresholds per class
    thresholds = find_classwise_thresholds_by_youden(y_val, y_prob)
    
    # Convert probabilities into binary decisions based on thresholds
    y_prob_bin = custom_classify(y_prob, thresholds)
    y_pred = y_prob_bin.argmax(axis=1)

    metrics, cm = get_metrics(y_val, y_pred, y_prob)

    # Add current weight setting to metrics
    metrics['GN_weight(class_1)'] = a
    metrics['GP_weight(class_2)'] = b

    results.append(metrics)

100%|██████████| 20/20 [06:56<00:00, 20.80s/it]


In [8]:
results_df = pd.concat(results, ignore_index=False)
results_df = results_df.reset_index(names='Class')
results_df

,Class,AUROC,AUPRC,Accuracy,F1 Score,Sensitivity,Specificity,PPV,NPV,a,b
0,Class 0,0.844874,0.974767,0.699090,0.802299,0.683242,0.832237,0.971604,0.238230,1,1
1,Class 1,0.858273,0.397076,0.841148,0.366806,0.595023,0.861775,0.265121,0.962108,1,1
2,Class 2,0.767251,0.095774,0.798810,0.114022,0.445783,0.809369,0.065371,0.979930,1,1
3,Class 0,0.840063,0.973804,0.676522,0.784222,0.657792,0.833882,0.970818,0.224834,1,4
4,Class 1,0.858318,0.404799,0.830301,0.350736,0.592760,0.850209,0.249049,0.961407,1,4
5,Class 2,0.723650,0.079639,0.785339,0.103725,0.427711,0.796036,0.059019,0.978950,1,4
6,Class 0,0.842584,0.974054,0.668824,0.777582,0.647807,0.845395,0.972377,0.222222,1,8
7,Class 1,0.861905,0.401911,0.874388,0.361210,0.459276,0.909177,0.297654,0.952523,1,8
8,Class 2,0.744169,0.082305,0.717810,0.102393,0.554217,0.722703,0.056407,0.981885,1,8
9,Class 0,0.832146,0.972363,0.680021,0.787103,0.661903,0.832237,0.970715,0.226601,1,16


## **CBC/CPD vs CBC vs CPD**

#### **Assign variables**

In [9]:
# Extract target labels from each dataset
y = data_train.loc[:, 'final_label']
y_test = data_test.loc[:, 'final_label']
y_wk = data_WMH.loc[:, 'final_label']
y_an = data_ANH.loc[:, 'final_label']

# Extract full feature set (columns 6 to 92) from each dataset
x = data_train.iloc[:, 6:93]
x_test = data_test.iloc[:, 6:93]
x_wk = data_WMH.iloc[:, 6:93]
x_an = data_ANH.iloc[:, 6:93]

# Extract features of CBC (columns 0 to 30) 
x_cbc = x.iloc[:, 0:31]
x_test_cbc = x_test.iloc[:, 0:31]
x_wk_cbc = x_wk.iloc[:, 0:31]
x_an_cbc = x_an.iloc[:, 0:31]

# Extract features of CPD (columns 31 onward)
x_cpd = x.iloc[:, 31:]
x_test_cpd = x_test.iloc[:, 31:]
x_wk_cpd = x_wk.iloc[:, 31:]
x_an_cpd = x_an.iloc[:, 31:]

#### **Start training**

In [11]:
class_weights = {0: 1, 1: 1, 2:1}  # Define class weights for multi-class classification

# Define different input feature sets
input_sets = {
    "All": x,        # All available features
    "CBC": x_cbc,    # CBC-related features 
    "CPD": x_cpd     # CPD-related features 
}

results = {}         # Store performance metrics
y_prob_dict = {}     # Store predicted probabilities
y_val_dict = {}      # Store ground truth labels for validation sets

# Loop over each feature set
for name, x_input in input_sets.items():
    print(name)

    # Split the dataset using GroupShuffleSplit (ensure samples from same 'ID' stay together)
    gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)
    train_idx, val_idx = next(gss.split(x_input, y, groups=data_train['ID']))
    x_train, x_val = x_input.iloc[train_idx], x_input.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    y_val_bin = label_binarize(y_val, classes=[0, 1, 2])  # Convert labels to one-hot format

    # Preprocessing
    x_train_imp, m_mean = imputer(x_train, impute_method, train=True)
    x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

    x_val_imp = imputer(x_val, impute_method, train=False, num_impute_values=m_mean)
    x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)

    # Train the model
    model = CatBoostClassifier(verbose=False, class_weights=class_weights)
    model.fit(x_train_scale, y_train)

    # Predict class probabilities on the validation set
    y_prob = model.predict_proba(x_val_scale)
    y_prob_dict[name] = y_prob
    y_val_dict[name] = y_val

    # Determine optimal thresholds using Youden’s index (for multi-class)
    thresholds = find_classwise_thresholds_by_youden(y_val, y_prob)

    # Apply custom classification based on thresholds
    y_prob_bin = custom_classify(y_prob, thresholds)
    y_pred = y_prob_bin.argmax(axis=1)

    metrics , cm = get_metrics(y_val, y_pred, y_prob)

    for i in range(3):
        class_label = f"Class {i}"
        auroc = metrics.loc[class_label, "AUROC"]
        auprc = metrics.loc[class_label, "AUPRC"]
        print(f"{class_label}: AUROC = {auroc:.3f}, AUPRC = {auprc:.3f}")


All
Class 0: AUROC = 0.845, AUPRC = 0.975
Class 1: AUROC = 0.858, AUPRC = 0.397
Class 2: AUROC = 0.767, AUPRC = 0.096
CBC
Class 0: AUROC = 0.822, AUPRC = 0.972
Class 1: AUROC = 0.830, AUPRC = 0.334
Class 2: AUROC = 0.755, AUPRC = 0.095
CPD
Class 0: AUROC = 0.831, AUPRC = 0.972
Class 1: AUROC = 0.844, AUPRC = 0.390
Class 2: AUROC = 0.725, AUPRC = 0.084


- save the result for Delong test

In [ ]:
dump(y_prob_dict, rf'{cache_dir}\prob_for_Delong_var.joblib')
dump(y_val_dict, rf'{cache_dir}\y_true_for_Delong_var.joblib')

['d:\\python_code\\CPD\\GS\\cache\\y_true_for_Delong_var.joblib']

#### **Delong test**

- Load the result for Delong test

In [12]:
y_prob_dict = load( rf'{cache_dir}\prob_for_Delong_var.joblib')
y_val_dict =  load( rf'{cache_dir}\y_true_for_Delong_var.joblib')

In [15]:
for k in range(3):  # Loop over class index 0 (nonbacteremia), 1 (gram negative), 2 (gram positive)
    print(f"\n=== Class {k} ===")
    
    # Create binary labels (1 for the current class, 0 for others)
    y_true_binary = (y_val_dict["All"] == k).astype(int)
    
    # Extract predicted probabilities for the current class from each model
    y_score_all = y_prob_dict["All"][:, k]
    y_score_cbc = y_prob_dict["CBC"][:, k]
    y_score_cpd = y_prob_dict["CPD"][:, k]

    # Perform DeLong test to compute the p-value for AUROC differences
    # np.exp is applied because the function returns log(p), so we convert it back
    p_value_all_cbc = np.exp(delong_roc_test(y_true_binary, y_score_all, y_score_cbc))[0][0]
    p_value_all_cpd = np.exp(delong_roc_test(y_true_binary, y_score_all, y_score_cpd))[0][0]
    p_value_cbc_cpd = np.exp(delong_roc_test(y_true_binary, y_score_cbc, y_score_cpd))[0][0]

    # Print the corrected p-values for pairwise AUROC comparisons
    print(f"Corrected p-value for All vs CBC: {p_value_all_cbc:.4f}")
    print(f"Corrected p-value for All vs CPD: {p_value_all_cpd:.4f}")
    print(f"Corrected p-value for CBC vs CPD: {p_value_cbc_cpd:.4f}")



=== Class 0 ===
Corrected p-value for All vs CBC: 0.0051
Corrected p-value for All vs CPD: 0.0732
Corrected p-value for CBC vs CPD: 0.5098

=== Class 1 ===
Corrected p-value for All vs CBC: 0.0016
Corrected p-value for All vs CPD: 0.0941
Corrected p-value for CBC vs CPD: 0.3190

=== Class 2 ===
Corrected p-value for All vs CBC: 0.6795
Corrected p-value for All vs CPD: 0.0787
Corrected p-value for CBC vs CPD: 0.4220


## **Imputation method**

In [17]:
imputation_methods = ['mean', 'zero', 'median', 'knn']  # Define different imputation methods
class_weights = {0: 1, 1: 1, 2:1}  # Define class weights for balanced multi-class classification

y_prob_dict = {}          # Store predicted probabilities
y_val_dict = {}           # Store ground truth labels
results_imp_method = {}   # Store performance metrics by imputation method

# Split the dataset using GroupShuffleSplit to ensure fair comparison (same random seed)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=random_seed)
train_idx, val_idx = next(gss.split(x, y, groups=data_train['ID']))

# Extract training and validation sets
x_train, x_val = x.iloc[train_idx], x.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

# Loop through different imputation methods
for method in imputation_methods:
    print(method)
    
    # Convert multiclass labels to binary format for AUROC/AUPRC calculations
    y_val_bin = label_binarize(y_val, classes=[0, 1, 2])

    # Impute missing values using the selected method
    x_train_imp, m_mean = imputer(x_train, method, train=True)
    x_train_scale, train_scaler = scaling(x_train_imp, None, train=True)

    x_val_imp = imputer(x_val, method, train=False, num_impute_values=m_mean)
    x_val_scale, _ = scaling(x_val_imp, train_scaler, train=False)

    # Train the CatBoost classifier
    model = CatBoostClassifier(verbose=False, class_weights=class_weights)
    model.fit(x_train_scale, y_train)

    # Predict class probabilities
    y_prob = model.predict_proba(x_val_scale)
    y_prob_dict[method] = y_prob
    y_val_dict[method] = y_val  # Store ground truth labels for evaluation
    
    # Determine optimal thresholds using Youden’s index (for multi-class)
    thresholds = find_classwise_thresholds_by_youden(y_val, y_prob)
    
    # Apply threshold-based classification
    y_prob_bin = custom_classify(y_prob, thresholds)
    y_pred = y_prob_bin.argmax(axis=1)

    # Compute performance metrics
    metrics, cm = get_metrics(y_val, y_pred, y_prob)

    # Print AUROC and AUPRC per class
    for i in range(3):
        class_label = f"Class {i}"
        auroc = metrics.loc[class_label, "AUROC"]
        auprc = metrics.loc[class_label, "AUPRC"]
        print(f"{class_label}: AUROC = {auroc:.3f}, AUPRC = {auprc:.3f}")


mean
Class 0: AUROC = 0.845, AUPRC = 0.975
Class 1: AUROC = 0.858, AUPRC = 0.397
Class 2: AUROC = 0.767, AUPRC = 0.096
zero
Class 0: AUROC = 0.844, AUPRC = 0.975
Class 1: AUROC = 0.858, AUPRC = 0.400
Class 2: AUROC = 0.757, AUPRC = 0.093
median
Class 0: AUROC = 0.845, AUPRC = 0.975
Class 1: AUROC = 0.857, AUPRC = 0.400
Class 2: AUROC = 0.758, AUPRC = 0.093
knn
Class 0: AUROC = 0.845, AUPRC = 0.975
Class 1: AUROC = 0.857, AUPRC = 0.400
Class 2: AUROC = 0.769, AUPRC = 0.106


- save the result for Delong test

In [ ]:
dump(y_prob_dict, rf'{cache_dir}\prob_for_Delong_imp.joblib')
dump(y_val_dict, rf'{cache_dir}\y_true_for_Delong_imp.joblib')

['d:\\python_code\\CPD\\GS\\cache\\y_true_for_Delong_imp.joblib']

#### **Delong test**

- Load the result for Delong test

In [18]:
y_prob_dict = load( rf'{cache_dir}\prob_for_Delong_imp.joblib')
y_val_dict =  load( rf'{cache_dir}\y_true_for_Delong_imp.joblib')

In [19]:
import itertools

k = 1  # Select the target class to compare (e.g., Class 1 = Gram-negative bacteremia)
y_true_binary = (y_val_dict['mean'] == k).astype(int)  # Use any imputation method, since data split is fixed

# Extract predicted probabilities for the selected class from each method
method_scores = {
    method: y_prob[:, k] for method, y_prob in y_prob_dict.items()
}

# Store p-value results
delong_results = []

# Generate all pairwise combinations of imputation methods
for method1, method2 in itertools.combinations(method_scores.keys(), 2):
    score1 = method_scores[method1]
    score2 = method_scores[method2]

    # Perform DeLong test to assess AUROC difference
    p_val_log = delong_roc_test(y_true_binary, score1, score2)  # result in log(p)
    p_val = np.exp(p_val_log)[0][0]  # Convert back to raw p-value

    # Append results
    delong_results.append({
        'Method 1': method1,
        'Method 2': method2,
        'Class': f'Class {k}',
        'p-value': round(p_val, 4)
    })

# Display results as a DataFrame
df_delong = pd.DataFrame(delong_results)
df_delong


,Method 1,Method 2,Class,p-value
0,mean,zero,Class 1,0.9919
1,mean,median,Class 1,0.8776
2,mean,knn,Class 1,0.7657
3,zero,median,Class 1,0.8937
4,zero,knn,Class 1,0.7977
5,median,knn,Class 1,0.9018
